In [129]:
import pandas as pd

In [130]:
pkl='allyears_with_teams'
df= pd.read_pickle(pkl)

In [131]:
#Create lists of players on each team
with open('euro_players_scrape.txt', 'r') as rf:
    euro_players=[]
    for line in rf:
        n=line.split('  ')
        euro_players.append(n[0])
with open('am_players_scrape.txt', 'r') as rf:
    am_players=[]
    for line in rf:
        n=line.split('  ')
        am_players.append(n[0])
        
#remove non-playing captains and fix oscar's name
am_players.remove('Ã“scar DomÃ\xadnguez')
am_players.append('Óscar Domínguez')
am_players.remove('Buddy Hall')
am_players.remove('Don Wardell ')
euro_players.remove('Johan Ruijsink')

In [132]:
#create player dictionary to boolean index df by player
#*note: str.contains allows matches for when there are other characters as well, such as in doubles 
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [133]:
#Create function for boolean indexing the dataframe
#will eventually use this to try out decorators to simplify the win loss code
def boolind(df = df, d = d,**kwargs):
    if 'form' in kwargs.keys():
        df = df[df['Format']==kwargs['form']]
    if 'Europe_won' in kwargs.keys():
        df = df [df['Europe_won']==kwargs['Europe_won']]
    if 'player' in kwargs.keys():
        df = df [d [  kwargs['player']  ] ]
    if 'year' in kwargs.keys():
        df = df.loc[kwargs['year']]
    return(df)

In [134]:
df[df['Format']=='Trebles']

Format                              European_player  \
Year Match                                                         
2006 1      Trebles      Mika Immonen David Alcaide Ralf Souquet   
     2      Trebles  Imran Majid Thomas Engert Nick van den Berg   

           European_score American_score  \
Year Match                                 
2006 1                  6              3   
     2                  2              6   

                                      American_player Europe_won  
Year Match                                                        
2006 1          Johnny Archer John Schmidt Mike Davis       True  
     2      Corey Deuel Earl Strickland Rodney Morris      False

In [135]:
#change trebles to doubles
df.loc[(2006,1),'Format']='Triples'
df.loc[(2006,2),'Format']='Triples'

In [136]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
triples = df['Format']=='Triples'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [137]:
def winloss(player, country):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(swin,sloss)
    spct=round(swin*100/(swin+sloss),2)

    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(dwin,dloss)
    dpct=round(dwin*100/(dwin+dloss),2)
    
        #triples
    if 'a' in country:
        trwin=df[d[player]&america_won&triples].shape[0]
        trloss=df[d[player]&~america_won&triples].shape[0]
    else:
        trloss=df[d[player]&america_won&triples].shape[0]
        trwin=df[d[player]&~america_won&triples].shape[0] 
    trwl='{}-{}'.format(trwin,trloss)
    trpct=round(trwin*100/(trwin+trloss),2)
    
    #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
    twl='{}-{}'.format(twin,tloss)
    tpct=round(twin*100/(twin+tloss),2)
    
    #combined
    
    cwin=dwin+swin
    closs=dloss+sloss
    cwl='{}-{}'.format(cwin,closs)
    cpct=round(cwin*100/(cwin+closs),2)
    print("singles: {}   spct:{}%   doubles: {}  dpct:{}% twl: {} trwl:{}   combined: {}  cpct:{}%".format(swl,spct,dwl,dpct,twl,trwl,cwl,cpct))
    return(swl,dwl,cwl)

In [138]:
winloss('Earl Strickland','am')

singles: 14-12   spct:53.85%   doubles: 24-12  dpct:66.67% twl: 3-2 trwl:2-0   combined: 38-24  cpct:61.29%


('14-12', '24-12', '38-24')

In [160]:
def winlosslist(player, country):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(swin,sloss)
    try:
        spct=swin*100/(swin+sloss)
    except:
        spct=0
    
    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(dwin,dloss)
    try:
        dpct=dwin*100/(dwin+dloss)   
    except:
        dpct=0
    
        #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
    twl='{}-{}'.format(twin,tloss)
    try:
        tpct=twin*100/(twin+tloss)   
    except:
        tpct=0
        
        #triples
    if 'a' in country:
        trwin=df[d[player]&america_won&triples].shape[0]
        trloss=df[d[player]&~america_won&triples].shape[0]
    else:
        trloss=df[d[player]&america_won&triples].shape[0]
        trwin=df[d[player]&~america_won&triples].shape[0] 
    trwl='{}-{}'.format(trwin,trloss)
    try:
        trpct=round(trwin*100/(trwin+trloss),2)        
    except:
        trpct=0

    #combined
    cwin=dwin+swin+twin
    closs=dloss+sloss+tloss
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,swin,sloss,swl,round(spct,2),dwin,dloss,dwl,round(dpct,2), twl,trwl, cwin,closs,cwl,round(cpct,2)]
    return(l)

In [161]:
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        print('failure- am: ', player)
#         pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        print('failure -eu ', player)
#         pass

In [163]:
import pandas as pd
colz = 'Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)
player_stats.head(10)

,Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cwin,Closs,Cwl,Cpct
0,Johnny Archer,America,17,13,17-13,56.67,27,23,27-23,54.00,5-4,1-1,49,40,49-40,55.06
1,Earl Strickland,America,14,12,14-12,53.85,24,12,24-12,66.67,3-2,2-0,41,26,41-26,61.19
2,Rodney Morris,America,10,8,10-8,55.56,12,17,12-17,41.38,3-5,2-0,25,30,25-30,45.45
3,Shane Van Boening,America,10,15,10-15,40.00,11,14,11-14,44.00,4-6,0-0,25,35,25-35,41.67
4,Corey Deuel,America,7,6,7-6,53.85,13,12,13-12,52.00,5-2,2-0,25,20,25-20,55.56
5,Jeremy Jones,America,7,7,7-7,50.00,10,9,10-9,52.63,0-1,0-0,17,17,17-17,50.00
6,Charlie Williams,America,6,2,6-2,75.00,6,7,6-7,46.15,0-0,0-0,12,9,12-9,57.14
7,Nick Varner,America,3,4,3-4,42.86,5,7,5-7,41.67,0-0,0-0,8,11,8-11,42.11
8,Mike Dechaine,America,1,6,1-6,14.29,2,8,2-8,20.00,2-2,0-0,5,16,5-16,23.81
9,Dennis Hatch,America,4,3,4-3,57.14,5,7,5-7,41.67,2-2,0-0,11,12,11-12,47.83


In [164]:
import pickle
pd.to_pickle(player_stats,'player_stats')

In [165]:
def playerinfo(player,country):
    '''Get list of info about player stats and output every match they played'''
    wl=winlosslist(player,country)
    frame=df[d[player]]
    print((winlosslist(player,country)))
    return(frame)

In [166]:
stat_l = []
for player in am_players:
    try:
        unit = winlosslist(player,'America')
        stat_l.append(unit)
    except:
        pass
for player in euro_players:
    try:
        unit = winlosslist(player,'Europe')
        stat_l.append(unit)
    except:
        pass

Data including 2016

Team USA
• Shane Van Boening (10 years -- 2007-2016) ..... 10-15 singles, 11-14 doubles, 4-6 teams, 25-35 total (winning percentage of 42%)

• Rodney Morris (10 years -- 2003-2008, 2010-2011, 2013, 2016) ..... 10-8 singles, 12-17 doubles, 2-0 trebles, 3-5 teams, 27-30 total (47%)

• Mike Dechaine (4 years -- 2011-2012, 2015-2016) ..... 1-6 singles, 2-8 doubles, 2-2 teams, 5-16 total (24%)

• Justin Bergman (3 years -- 2014-2016) ..... 1-5 singles, 4-3 doubles, 1-2 teams, 6-10 total (38%)

• Skyler Woodward (2 years -- 2015-2016) ..... 0-3 singles, 3-3 doubles, 1-1 teams, 4-7 total (36%)

• Team USA Total ..... 22-37 singles, 32-45 doubles, 2-0 trebles, 11-16 teams, 67-98 total (41%)

Team Europe
• Niels Feijen (12 years -- 2001, 2004-2005, 2007-2009, 2011-2016) ..... 12-8 singles, 24-9 doubles, 5-4 teams, 41-21 total (66%)

• Darren Appleton (8 years -- 2009-2016) ..... 10-4 singles, 10-15 doubles, 5-3 teams, 25-22 total (53%)

• Mark Gray (3 years -- 2008, 2014, 2016) ..... 1-1 singles, 4-5 doubles, 3-0 teams, 8-6 total (57%)

• Albin Ouschan (2 years -- 2015-2016) ..... 3-0 singles, 5-1 doubles, 1-1 teams, 9-2 total (82%)

• Jayson Shaw (1 year -- 2016) ..... 1-1 singles, 1-1 doubles, 1-0 teams, 3-2 total (60%)

• Team Europe Total ..... 27-14 singles, 44-31 doubles, 15-8 teams, 86-53 total (62%)

In [ ]:
usa = ['Shane Van Boening', 'Rodney Morris', 'Mike Dechaine', 'Justin Bergman', 'Skyler Woodward']
for player in usa:
    print(winlosslist(player, 'am'))

Team USA
• Shane Van Boening 10-15 singles, 11-14 doubles, 4-6 teams, 25-35 total (42%)

• Rodney Morris 10-8 singles, 12-17 doubles, 2-0 trebles, 3-5 teams, 27-30 total (47%)

• Mike Dechaine 1-6 singles, 2-8 doubles, 2-2 teams, 5-16 total (24%)

• Justin Bergman  1-5 singles, 4-3 doubles, 1-2 teams, 6-10 total (38%)

• Skyler Woodward  0-3 singles, 3-3 doubles, 1-1 teams, 4-7 total (36%)


Team Europe
• Niels Feijen 12-8 singles, 24-9 doubles, 5-4 teams, 41-21 total (66%)

• Darren Appleton  10-4 singles, 10-15 doubles, 5-3 teams, 25-22 total (53%)

• Mark Gray   1-1 singles, 4-5 doubles, 3-0 teams, 8-6 total (57%)

• Albin Ouschan  3-0 singles, 5-1 doubles, 1-1 teams, 9-2 total (82%)

• Jayson Shaw 1-1 singles, 1-1 doubles, 1-0 teams, 3-2 total (60%)

In [ ]:
eu = ['Darren Appleton', 'Mark Gray', 'Albin Ouschan', 'Niels Feijen']
for player in eu:
    print(winlosslist(player, 'u'))

#More info to verify:
• Johnny Archer (17 years) ..... 18-13 singles, 26-23 doubles, 0-2 trebles, 5-4 teams, 49-42 total (winning percentage of 54%)
• Earl Strickland (14 years) ..... 14-12 singles, 24-12 doubles, 1-1 trebles, 3-2 teams, 42-27 total (61%)
• Corey Deuel (9 years) ..... 7-6 singles, 13-12 doubles, 2-0 trebles, 5-2 teams, 27-20 total (57%)
And more:
Dennis Hatch (4 years -- 2009, 2010, 2012, 2013) ..... 4-3 singles, 5-7 doubles, 2-2 teams, 11-12 total (winning percentage 48%)

In [ ]:
pl=['Johnny Archer','Earl Strickland','Corey Deuel','Dennis Hatch']
for player in pl:
    print(winlosslist(player,'am'))

In [ ]:
import pickle

In [ ]:
pickle.dump(winlosslist, open('winlosspkl','wb'))

# Data Analysis

In [187]:
#Best combined win percentage
player_stats = pd.DataFrame(data=stat_l, columns = colz)
player_stats[['Player','Cwl','Cpct']].sort_values('Cpct',ascending=False).head(11).reset_index(drop=True)

#Most singles wins
player_stats[['Player','Swl','Swin']].sort_values('Swin',ascending=False).head(11).reset_index(drop=True)

#Best singles win percentage
player_stats[['Player','Swin','Swl','Spct']].sort_values(['Spct','Swin'],ascending=[False,False]).head().reset_index(drop=True)

#Most doubles wins
player_stats[['Player','Dwl','Dwin']].sort_values('Dwin',ascending=False).reset_index(drop=True).head(10)

#Best doubles win percentage

,Player,Cwl,Cpct
0,Vivian Villarreal,5-0,100.00
1,Jeanette Lee,5-0,100.00
2,Albin Ouschan,9-2,81.82
3,Gabe Owen,4-1,80.00
4,C.J. Wiley,4-1,80.00
5,Karl Boyes,17-5,77.27
6,Shannon Daulton,7-3,70.00
7,Mike Davis,4-2,66.67
8,Niels Feijen,41-21,66.13
9,Tom Storm,7-4,63.64


,Player,Swl,Swin
0,Ralf Souquet,20-13,20
1,Johnny Archer,17-13,17
2,Mika Immonen,16-17,16
3,Steve Davis,14-13,14
4,Earl Strickland,14-12,14
5,Niels Feijen,12-8,12
6,Oliver Ortmann,11-10,11
7,Rodney Morris,10-8,10
8,Darren Appleton,10-4,10
9,Shane Van Boening,10-15,10


,Player,Swin,Swl,Spct
0,Chris Melling,5,5-0,100.0
1,Reed Pierce,4,4-0,100.0
2,Vivian Villarreal,3,3-0,100.0
3,Albin Ouschan,3,3-0,100.0
4,Jeanette Lee,2,2-0,100.0


,Player,Dwl,Dwin
0,Johnny Archer,27-23,27
1,Ralf Souquet,27-17,27
2,Earl Strickland,24-12,24
3,Niels Feijen,24-9,24
4,Mika Immonen,22-17,22
5,Corey Deuel,13-12,13
6,Rodney Morris,12-17,12
7,Shane Van Boening,11-14,11
8,Jeremy Jones,10-9,10
9,Darren Appleton,10-15,10


,Player,Dwl,Dpct
0,Vivian Villarreal,2-0,100.00
1,Dallas West,3-0,100.00
2,Bobby Hunter,3-0,100.00
3,Jeanette Lee,3-0,100.00
4,Lou Butera,2-0,100.00
5,Mark Wilson,2-0,100.00
6,C.J. Wiley,1-0,100.00
7,Vincent Facquet,1-0,100.00
8,Karl Boyes,10-2,83.33
9,Albin Ouschan,5-1,83.33
